With testing, make sure you have a good GPU available... CPU training on Colab takes forever, and I got Colab Pro to run all this on a GPU.
Also, even with multiple weeks of running data retrieval with yahoo finance (yfinance), I keep getting rate limited whenever I ask for more than a week of data from more than one ticker.
Using Kaggle dataset with historical options data.

In [ ]:
#imports
!pip install -q kaggle yfinance torch-geometric-temporal torch-geometric

import os, json, math, datetime as dt, numpy as np, pandas as pd, yfinance as yf
import torch, torch.nn as nn
from torch_geometric_temporal.nn.recurrent import GConvGRU
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.metrics import confusion_matrix

#kaggle - may need to change in future
os.makedirs("/root/.kaggle", exist_ok=True)
with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(json.dumps({"username":"<YOUR_ID>","key":"<YOUR_KEY>"}))  # swap in env vars or secrets
os.chmod("/root/.kaggle/kaggle.json", 0o600)

DATASET = "jacksoncrow/stock-market-dataset"
!kaggle datasets download -d $DATASET -p ./jc_data -q
!unzip -q -o ./jc_data/stock-market-dataset.zip -d ./jc_data

#data
TICKERS = ["AAPL","JNJ","JPM","AMZN","KO","XOM","BA","NEE","GOOG","LIN"]
START, END = "2022-01-01", "2023-12-31"

def load_stock_csv(ticker):
    path = f"./jc_data/price/{ticker}.us.txt"
    df = pd.read_csv(path, parse_dates=["date"])
    df = df[(df["date"]>=START)&(df["date"]<=END)].copy()
    df.rename(columns={"date":"Date","open":"Open","high":"High",
                       "low":"Low","close":"Close","volume":"Volume"}, inplace=True)
    df.sort_values("Date", inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

#Feature eng.
def add_indicators(df):
    df["MA10"] = df["Close"].rolling(10).mean()
    df["MA50"] = df["Close"].rolling(50).mean()
    df["Return"] = df["Close"].pct_change()
    df["Volatility"] = df["Return"].rolling(30).std()

    # RSI(14)
    delta = df["Close"].diff()
    gain, loss = delta.clip(lower=0), -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df["RSI"] = 100 - (100 / (1 + rs))

    # MACD
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = ema12 - ema26
    df["MACD_signal"] = df["MACD"].ewm(span=9, adjust=False).mean()

    # Bollinger
    mavg20 = df["Close"].rolling(20).mean()
    mstd20 = df["Close"].rolling(20).std()
    df["BB_up"] = mavg20 + 2*mstd20
    df["BB_dn"] = mavg20 - 2*mstd20
    return df

stocks = {t: add_indicators(load_stock_csv(t)) for t in TICKERS}

#VIX
#Yfinance keeps rate limiting! Don't use!
#vix = yf.download("^VIX", start=START, end=END, progress=False)["Adj Close"].rename("VIX")

#trades
np.random.seed(343)
def choose_strategy():
    return np.random.choice(["random","momentum"])

trades = []
for ticker, df in stocks.items():
    months = pd.period_range(START, END, freq="M")
    for m in months:
        month_df = df[df["Date"].dt.to_period("M")==m]
        if len(month_df) < 22:         # skip very short months (holidays)
            continue
        buy_day = month_df.iloc[0]
        exp_day = month_df.iloc[-1]
        strategy = choose_strategy()

        #exercise date
        if strategy=="random":
            sell_day = month_df.sample(1, random_state=7).iloc[0]
        else:  # momentum: exit on first bearish crossover else hold
            sell_day = exp_day
            short_ma = month_df["MA10"]
            long_ma  = month_df["MA50"]
            cross = np.where(short_ma < long_ma)[0]
            if len(cross)>0:
                sell_day = month_df.iloc[cross[0]]

        #rbos
        fut_max = month_df[month_df["Date"]>sell_day["Date"]]["Close"].max()
        fut_max = fut_max if not pd.isna(fut_max) else sell_day["Close"]
        rbos = (fut_max - sell_day["Close"]) / sell_day["Close"]

        trades.append({
            "ticker":     ticker,
            "strategy":   strategy,
            "buy_date":   buy_day["Date"],
            "sell_date":  sell_day["Date"],
            "exp_date":   exp_day["Date"],
            "buy_price":  buy_day["Close"],
            "sell_price": sell_day["Close"],
            "max_future": fut_max,
            "RBOS":       rbos,
            "binary_regret": int(rbos > 0.01),
            "opp_regret":    int(rbos > 0.10),
            "delta_regret":  int(rbos > 0.20)
        })

trades.sort(key=lambda x: x["sell_date"])

#graph snapshots
asset_idx = {t:i+2 for i,t in enumerate(TICKERS)}  # node 0 = Trader, 1 = Option
snap_feats, snap_edges, snap_eweights, snap_labels = [], [], [], []

for tr in trades:
    # --- node features ---
    N = 2 + len(TICKERS)
    feat_dim = 11  # 1(type) + 10 numeric
    X = torch.zeros((N, feat_dim), dtype=torch.float32)

    # Trader node (index 0)
    X[0,0] = 1
    X[0,1] = 0 if tr["strategy"]=="random" else 1  # simple one‑hot strategy

    # Option node (index 1)
    stock_df = stocks[tr["ticker"]]
    row = stock_df[stock_df["Date"]==tr["sell_date"]].iloc[0]
    S = row["Close"]
    K = S * 1.0                       # assume ATM strike
    tau = (tr["exp_date"] - tr["sell_date"]).days
    iv = row["Volatility"]*math.sqrt(252) if not np.isnan(row["Volatility"]) else 0
    X[1,:] = torch.tensor([2, S/K, tau/252, iv, 0,0,0,0,0,0,0])

    # Asset nodes
    for tk, df in stocks.items():
        idx = asset_idx[tk]
        r = df[df["Date"]==tr["sell_date"]]
        if r.empty: continue
        r = r.iloc[0]
        vix_val = vix.loc[tr["sell_date"]] if tr["sell_date"] in vix.index else np.nan
        X[idx] = torch.tensor([
            3,
            r["MA10"] or 0, r["MA50"] or 0, r["RSI"] or 0,
            r["MACD"] or 0, r["Return"] or 0, r["Volatility"] or 0,
            r["BB_up"] or 0, r["BB_dn"] or 0, vix_val or 0
        ])

    #edges (Trader to/from Option, Option to/from Asset)
    edges = []
    eweights = []

    # Trader‑Option
    edges += [[0,1],[1,0]]
    eweights += [1,1]   # weight 1 (no volume meaning)

    # Option‑Asset (use normalized volume as weight)
    asset_node = asset_idx[tr["ticker"]]
    vol_norm = (row["Volume"] / (stock_df["Volume"].max()+1e-9))
    edges += [[1,asset_node],[asset_node,1]]
    eweights += [vol_norm]*2

    snap_feats.append(X)
    snap_edges.append(torch.tensor(edges, dtype=torch.long).t().contiguous())
    snap_eweights.append(torch.tensor(eweights, dtype=torch.float32))
    snap_labels.append(torch.tensor([tr["binary_regret"],
                                     tr["opp_regret"],
                                     tr["delta_regret"]], dtype=torch.float32))

#GNN model
class RegretGNN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super().__init__()
        self.gru = GConvGRU(in_dim, hid_dim, K=2)
        self.fc  = nn.Linear(hid_dim, out_dim)
        self.h = None
    def forward(self, x, edge_index, edge_w):
        self.h = self.gru(x, edge_index, edge_w, self.h)
        # use hidden of Option node (index 1)
        return self.fc(self.h[1])

in_dim = snap_feats[0].shape[1]
model = RegretGNN(in_dim, 32, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion  = nn.BCEWithLogitsLoss()

#train/test split
T = len(snap_feats)
train_cut = int(0.7*T)
train_ids = list(range(train_cut))
test_ids  = list(range(train_cut, T))

#training
loss_curve = []
for epoch in range(100):
    model.train(); model.h=None
    total = 0
    for i in train_ids:
        optimizer.zero_grad()
        yhat = model(snap_feats[i], snap_edges[i], snap_eweights[i])
        loss = criterion(yhat, snap_labels[i])
        loss.backward()
        optimizer.step()
        total += loss.item()
    loss_curve.append(total/len(train_ids))

#eval
model.eval(); model.h=None
y_true, y_pred = [], []
with torch.no_grad():
    for j in test_ids:
        logits = model(snap_feats[j], snap_edges[j], snap_eweights[j])
        probs  = torch.sigmoid(logits)
        y_true.append(snap_labels[j].numpy().astype(int))
        y_pred.append((probs.numpy() >= 0.5).astype(int))
y_true, y_pred = np.array(y_true), np.array(y_pred)

#plots
plt.plot(loss_curve); plt.title("Training BCE"); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.show()

for k,name in enumerate(["Binary","Opportunity","Delta"]):
    cm = confusion_matrix(y_true[:,k], y_pred[:,k])
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Pred0","Pred1"], yticklabels=["Actual0","Actual1"])
    plt.title(f"{name} Regret"); plt.show()
